In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision import transforms
import torchvision

In [25]:
in_channels = 3
num_classes = 10
learning_rate = 0.001
batch_size = 1024
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<figure>
  <img src="asset/vgg16.png" alt="Grouped conv" width="600">
  <figcaption>VGG16 Architecture</figcaption>
</figure>

In [26]:
import sys
model = torchvision.models.vgg16(pretrained=True)  # download the pretrained weights
print(model)    

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [34]:
for param in model.parameters():
    param.requires_grad = False

In [35]:
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=num_classes)
)
    

In [28]:
# Show the last 10 immediate layers from features + avgpool + classifier
items = [(f"features.{i}", m) for i, m in enumerate(model.features)]
items += [("avgpool", model.avgpool)]
items += [(f"classifier.{i}", m) for i, m in enumerate(model.classifier)]

start = max(0, len(items) - 10)
for idx, (name, layer) in enumerate(items[start:], start):
    print(f"{idx}: {name}: {layer}")

25: features.25: ReLU(inplace=True)
26: features.26: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
27: features.27: ReLU(inplace=True)
28: features.28: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
29: features.29: ReLU(inplace=True)
30: features.30: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
31: avgpool: Identity()
32: classifier.0: Linear(in_features=512, out_features=256, bias=True)
33: classifier.1: ReLU()
34: classifier.2: Linear(in_features=256, out_features=10, bias=True)


In [29]:
# Transform: Convert image to tensor and normalize (0–1 range)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))   # normalize to mean=0.5, std=0.5
])

# Download MNIST training & test datasets
train_dataset = datasets.CIFAR10(
    root='./data', train=True, transform=transform, download=True
)
test_dataset = datasets.CIFAR10(
    root='./data', train=False, transform=transform, download=True
)

In [30]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
print('Finished Training')